# Statistical testing for CV-axis gene expression

**Pinned Environment:** [`envs/sc-spatial.yaml`](../../envs/sc-spatial.yaml)  

In [ ]:
import sys
from pathlib import Path
import os
import scanpy as sc
from sklearn.metrics import auc
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import ks_2samp
from statsmodels.stats.multitest import multipletests

In [ ]:
sys.path.append(str(Path.cwd().resolve().parents[1]))

from config.paths import BASE_DIR

h5ad_dir = BASE_DIR / "data/h5ad/export_10"
output_dir = BASE_DIR / "data/cv_axis/stats"

input_data = h5ad_dir / "iec-subset-resolvi-cc-v2.h5ad"

h5ad_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
adata = sc.read_h5ad(input_data)
adata

# Run KS

In [ ]:
results = []

group_col = "group"

group = adata.obs[group_col].values

for gene in tqdm(adata.var_names, desc="Global KS test"):
    expr = (
        adata[:, gene].X.toarray().flatten()
        if hasattr(adata[:, gene].X, "toarray")
        else adata[:, gene].X.flatten()
    )

    # Control group
    mask_ctrl = group == "Control"
    expr_ctrl = expr[mask_ctrl]

    # Treatment group
    mask_treat = group == "Trpv1-cre"
    expr_treat = expr[mask_treat]

    # Skip if insufficient
    if len(expr_ctrl) < 2 or len(expr_treat) < 2:
        continue

    # KS test
    ks_stat, pval = ks_2samp(expr_ctrl, expr_treat)

    results.append(
        {
            "gene": gene,
            "ks_stat": ks_stat,
            "pval": pval,
            "neglog10p": -np.log10(pval + 1e-10),
        }
    )

# Create df
ks_results_df = pd.DataFrame(results)

In [ ]:
# fdr
_, padj, _, _ = multipletests(ks_results_df["pval"], method="fdr_bh")
ks_results_df["padj"] = padj

In [ ]:
# Check number of genes tested
print(f"Total genes tested: {len(ks_results_df)}")
print(f"Total significant genes (padj < 0.05): {(ks_results_df['padj'] < 0.05).sum()}")

In [ ]:
ks_results_df

In [ ]:
sig_df = ks_results_df[ks_results_df["padj"] < 0.05]
print(f"Total significant genes (padj < 0.05): {len(sig_df)}")
print("Top 10 genes (sorted by padj):")
print(sig_df.sort_values("padj").head(10)[["gene", "padj", "ks_stat"]])

# Run delta auc

In [ ]:
def compute_global_delta_auc(
    adata, group_col="group", cv_col="crypt_villus_axis_scaled"
):
    results = []

    # Global variables
    cv_axis = adata.obs[cv_col].values
    group = adata.obs[group_col].values

    for gene in tqdm(adata.var_names, desc="Global delta AUC calculation"):
        expr = (
            adata[:, gene].X.toarray().flatten()
            if hasattr(adata[:, gene].X, "toarray")
            else adata[:, gene].X.flatten()
        )

        # Control group
        mask_ctrl = group == "Control"
        expr_ctrl = expr[mask_ctrl]
        cv_ctrl = cv_axis[mask_ctrl]
        if len(cv_ctrl) < 2:
            continue
        sorted_ctrl = np.argsort(cv_ctrl)
        auc_ctrl = auc(cv_ctrl[sorted_ctrl], expr_ctrl[sorted_ctrl])

        # Treatment group
        mask_treat = group == "Trpv1-cre"
        expr_treat = expr[mask_treat]
        cv_treat = cv_axis[mask_treat]
        if len(cv_treat) < 2:
            continue
        sorted_treat = np.argsort(cv_treat)
        auc_treat = auc(cv_treat[sorted_treat], expr_treat[sorted_treat])

        delta_auc = auc_treat - auc_ctrl

        if auc_ctrl > 0 and auc_treat > 0:
            log2_auc = np.log2(auc_treat / auc_ctrl)
            percent_change = (delta_auc / auc_ctrl) * 100
        else:
            log2_auc = np.nan
            percent_change = np.nan

        results.append(
            {
                "gene": gene,
                "auc_ctrl": auc_ctrl,
                "auc_treat": auc_treat,
                "delta_auc": delta_auc,
                "log2_auc": log2_auc,
                "percent_change_auc": percent_change,
            }
        )

    return pd.DataFrame(results)

In [ ]:
auc_results_df = compute_global_delta_auc(adata)

In [ ]:
auc_results_df

# Merge onto de df

In [ ]:
merged_df = pd.merge(ks_results_df, auc_results_df, on=["gene"], how="left")

In [ ]:
merged_df

In [ ]:
output_dir = os.path.join(base_dir, "data/cv_axis/stats")

In [ ]:
output_file = os.path.join(output_dir, "2025-05-01_epithelial-global-ks-auc.csv")
merged_df.to_csv(output_file)